In [1]:
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.util.common import *
from bigdl.dlframes.dl_classifier import *

from pyspark.sql.types import *
from pyspark.context import SparkContext

conf = create_spark_conf()
conf = conf.set("spark.executor.cores","1")
conf = conf.set("spark.cores.max","4")
conf = conf.setMaster("spark://spark-master-svc:7077")
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
sc

<SparkContext master=spark://spark-master-svc:7077 appName=pyspark-shell>

In [3]:
sqlContext = SQLContext(sc)

# Init BigDL
init_engine()

In [4]:
import bigdl
import bigdl.version
print(bigdl.version.__version__) # Get the current BigDL version
linear = Linear(2, 3) # Try to create a Linear layer

0.10.0
creating: createLinear


In [5]:
!wget https://s3-ap-southeast-1.amazonaws.com/analytics-zoo-models/object-detection/analytics-zoo_ssd-mobilenet-300x300_PASCAL_0.1.0.model

--2020-06-08 13:41:52--  https://s3-ap-southeast-1.amazonaws.com/analytics-zoo-models/object-detection/analytics-zoo_ssd-mobilenet-300x300_PASCAL_0.1.0.model
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.124.118
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.124.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23194692 (22M) [application/x-www-form-urlencoded]
Saving to: ‘analytics-zoo_ssd-mobilenet-300x300_PASCAL_0.1.0.model’

analytics-zoo_ssd-m 100%[===================>]  22.12M  5.38MB/s    in 4.1s    

2020-06-08 13:41:57 (5.38 MB/s) - ‘analytics-zoo_ssd-mobilenet-300x300_PASCAL_0.1.0.model’ saved [23194692/23194692]



In [18]:
from zoo.common.nncontext import init_nncontext
from zoo.models.image.objectdetection import *
import numpy
model = ObjectDetector.load_model("./analytics-zoo_ssd-mobilenet-300x300_PASCAL_0.1.0.model")
image_set = ImageSet.read("./benidorm2.jpg", sc)
output = model.predict_image_set(image_set)
config = model.get_config()
visualizer = Visualizer(config.label_map(), encoding="jpg")
visualized = visualizer(output).get_image(to_chw=False).collect()
img_id = "paco"
for img_id in range(len(visualized)):
        cv2.imwrite(output_path + '/' + str(img_id) + '.jpg', visualized[img_id])

creating: createVisualizer


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 7, 100.96.5.8, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 148, in load_stream
    yield self._read_with_length(stream)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 17

In [14]:
!hdfs dfs -put benidorm2.jpg benidorm2.jpg